# Import Statements

In [20]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import dominate
from dominate.tags import *
from dominate.svg import *
import espn_api.football
from espn_api.football import League, Player
import time
import pickle
import uuid
from typing import Literal

from python import constants, functions, espn_data

# Testing and Debugging

In [2]:
from IPython.core.display import HTML


def scale_value(value, data_min, data_max, window_max):
    '''
    Scales data points to 
    '''
    return round((value - data_min) / (data_max - data_min) * window_max, 3)

In [4]:
def df_to_svg(
        data: pd.DataFrame, 
        x_col: str, 
        y_col: str,
        chart_type: str = 'scatter',
        width: int = 500, 
        height: int = 300,
        x_tick_spacing: int = 50,
        y_tick_spacing: int = 50
):

    # margin x and y from edges of visual
    m_top, m_bottom, m_left, m_right = 10, 50, 60, 10
    # plot width and height
    P_x, P_y = width - m_left - m_right, height - m_top - m_bottom
    # tick margin (distance between axes and tick label)
    m_tick = 5

    # Bring in x and y ticks from calculate_ticks function
    include_zero = True if chart_type == 'bar' else False
    xlim, ylim = functions.calculate_limits(data=data, x_col=x_col, y_col=y_col, x_tick_spacing=x_tick_spacing, y_tick_spacing=y_tick_spacing, include_zero=include_zero)
    # if 'ylim' in kwargs:
    #     ylim = kwargs.pop('ylim')

    x_limit_min, x_limit_max = xlim
    y_limit_min, y_limit_max = ylim

    x_ticks, y_ticks = functions.calculate_ticks(xlim=xlim, ylim=ylim, x_tick_spacing=x_tick_spacing, y_tick_spacing=y_tick_spacing)

    d_svg = svg(
        xmlns='http://www.w3.org/2000/svg',
        width=width,
        height=height,
        viewBox=f'0 0 {width} {height}'
    )

    outer_group = g(
        font_family='Arial',
        font_size=10,
        text_anchor='middle',
        dominant_baseline='hanging'
    )
    d_svg.add(outer_group)

    border = functions.write_path([0, width, width, 0], [0, 0, height, height], close=True)
    border_path = path(d=border, fill='white', _id='border')
    outer_group.add(border_path)

    

    xlabel_group = g()

    grid_path_d = []
    for i, xtick in enumerate(x_ticks):
        if i == 0 or i == (len(x_ticks) - 1):
            continue
        x = round(m_left + (xtick - x_limit_min) / (x_limit_max - x_limit_min) * P_x, 3)

        gridline = functions.write_path([x, x], [m_top, height - m_bottom])
        grid_path_d.append(gridline)

        xtick_label = text(
            xtick,
            x=x,
            y=height - m_bottom + m_tick
        )
        xlabel_group.add(xtick_label)

    outer_group.add(xlabel_group)

    ylabel_group = g(
        text_anchor='end',
        dominant_baseline='middle'
    )
    zero_grid_path = False
    zero_ytick = False
    for ytick in y_ticks:
        y = round((height - m_bottom) - (ytick - y_limit_min) / (y_limit_max - y_limit_min) * P_y, 3)

        gridline = functions.write_path([m_left, width - m_right], [y, y])
        grid_path_d.append(gridline)

        ytick_label = text(
            ytick,
            x=m_left - m_tick,
            y=y
        )
        ylabel_group.add(ytick_label)


        if ytick == 0:
            zero_grid_path = path(d=gridline, stroke='black')
            zero_ytick = y

    outer_group.add(ylabel_group)

    grid_path = path(
        d=' '.join(grid_path_d),
        stroke='lightgrey'
    )
    outer_group.add(grid_path)
    if zero_grid_path:
        outer_group.add(zero_grid_path)

    axis_title_group = g(font_size=16)
    outer_group.add(axis_title_group)
    xlabel = text(
        x_col,
        x=(P_x / 2 + m_left),
        y=(height - (m_bottom / 2))
    )
    ylabel = text(
        y_col,
        x=10,
        y=(P_y / 2 + m_top),
        transform=f'rotate(-90, {10}, {P_y / 2 + m_top})'
    )

    axis_title_group.add([xlabel, ylabel])
    axes = functions.write_path([m_left, width - m_right, width - m_right, m_left], [m_top, m_top, height - m_bottom, height - m_bottom], close=True)
    axes_path = path(d=axes, fill='none', stroke='black', _id='axes')
    outer_group.add(axes_path)

    x_points = []
    y_points = []
    circles_group = g(
        stroke='black',
        stroke_width=1.5
    )
    bars_group = g(stroke='black')
    for index, row in data.iterrows():
        v_x = round(m_left + ((row[x_col] - x_limit_min) / (x_limit_max - x_limit_min) * (P_x)), 3)
        v_y = round((height - m_bottom) - ((row[y_col] - y_limit_min) / (y_limit_max - y_limit_min) * (P_y)), 3)

        bar_y = v_y if row[y_col] > 0 else zero_ytick
        bar_height = zero_ytick - v_y if row[y_col] > 0 else v_y - zero_ytick
        bar_width = 25

        bars_group.add(
            rect(
                x=v_x - bar_width / 2,
                y=bar_y,
                width=bar_width,
                height=bar_height,
                fill=constants.COLOR_DICT[row['Team'].lower()]
            )
        )
        circles_group.add(circle(cx=v_x, cy=v_y, r=4, fill=constants.COLOR_DICT[row['Team'].lower()]))
        
        x_points.append(v_x)
        y_points.append(v_y)


    if chart_type == 'line':
        line = functions.write_path(x_points, y_points)
        line_path = path(d=line, fill='none', stroke='black')
        outer_group.add(line_path)

    if chart_type in ['scatter','line']:
        outer_group.add(circles_group)

    if chart_type == 'bar':
        outer_group.add(bars_group)

    return d_svg



data = pd.concat([functions.summary_table(data=constants.GAME_DATA, year=2024, week=week) for week in range(1,15)])
data = data.loc[(data['Team'] == 'Haris')]

content=df_to_svg(
        data=data,
        x_col='Week',
        y_col='Luck Score',
        chart_type='line',
        x_tick_spacing=1,
        y_tick_spacing=2
    )

# print(content)
display(HTML(str(content)))


In [6]:
seasons = [functions.summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Andrew']

data

,Week,Year,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking
7,None,2019,Andrew,6,7,6-7,1574.14,1598.96,121.09,102,123.00,104,-1.91,-7,0,8
4,None,2020,Andrew,6,7,6-7,1645.84,1545.26,126.60,104,118.87,98,7.74,-2,0,5
0,None,2021,Andrew,10,4,10-4,1901.54,1755.20,135.82,110,125.37,101,10.45,-3,1,1
3,None,2022,Andrew,8,6,8-6,1599.88,1650.06,114.28,100,117.86,104,-3.58,-4,0,4
3,None,2023,Andrew,8,6,8-6,1672.62,1532.50,119.47,102,109.46,94,10.01,2,0,4
1,None,2024,Andrew,10,4,10-4,1873.18,1681.98,133.80,113,120.14,102,13.66,-3,0,2


In [ ]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

averages = {
    'Year':'Total',
    'Ranking':round(data['Ranking'].mean(), 2),
    'Points For':round(data['Points For'].mean(), 2),
    'Points Against':round(data['Points Against'].mean(), 2),
    'PF/G':round(data['PF/G'].mean(), 2),
    'PF/G+':round(data['PF/G+'].mean(), 2),
    'Avg Margin':round(data['Avg Margin'].mean(), 2),
    'Luck Score':round(data['Luck Score'].mean(), 2)
}

,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking,Year
8,Haris,5,8,5-8,1254.30,1304.58,96.48,97,100.35,101,-3.87,-5,0,9,2018
4,Haris,7,6,7-6,1548.30,1483.26,119.10,101,114.10,96,5.00,1,0,5,2019
9,Haris,3,10,3-10,1440.82,1773.88,110.83,91,136.45,112,-25.62,-13,0,10,2020
2,Haris,8,6,8-6,1855.22,1655.12,132.52,107,118.22,96,14.29,4,0,3,2021
6,Haris,7,7,7-7,1647.48,1579.54,117.68,103,112.82,99,4.85,-3,0,7,2022
0,Haris,12,2,12-2,1801.34,1456.30,128.67,110,104.02,89,24.65,10,1,1,2023
9,Haris,3,11,3-11,1468.60,1616.88,104.90,89,115.49,98,-10.59,1,0,10,2024


In [31]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

averages = [{
    'Year':'Total',
    'Wins':round(data['Wins'].mean(), 2),
    'Losses':round(data['Losses'].mean(), 2),
    'Ranking':round(data['Ranking'].mean(), 2),
    'Points For':round(data['Points For'].mean(), 2),
    'Points Against':round(data['Points Against'].mean(), 2),
    'PF/G':round(data['PF/G'].mean(), 2),
    'PF/G+':round(data['PF/G+'].mean(), 2),
    'Avg Margin':round(data['Avg Margin'].mean(), 2),
    'Luck Score':round(data['Luck Score'].mean(), 2)
}]

pd.DataFrame(averages)

,Year,Wins,Losses,Ranking,Points For,Points Against,PF/G,PF/G+,Avg Margin,Luck Score
0,Total,6.43,7.14,6.43,1573.72,1552.79,115.74,99.71,1.24,-0.71


In [32]:
def construct_dataframes(version: int) -> dict[str, pd.DataFrame]:
    '''
    Takes in espn-data.pkl file data and produces the following dataframes:
     - teams
     - drafts
     - matchups
     - games
     - player_games
     - players
    All of which will be converted into tables in the database

    Returns
    -------
    Dict : [Str, DataFrame]
    '''
    data = espn_data.read_pickle_file(version=version)

    data_teams = []
    data_drafts = []
    data_players = []
    data_matchups = []
    data_games = []
    data_player_games = []

    for year in data.keys():
        league = data[year]['League']
        players = data[year]['Players']
        box_scores = data[year]['Box Scores']

        for pick in league.draft:
            team_name = pick.team.owners[0]['firstName']
            if team_name == 'The':
                team_name = 'Klapp'
            elif team_name == 'Noah ':
                team_name = 'Noah'

            data_drafts.append(
                {
                    'draft_pick_id':str(uuid.uuid4()),
                    'player_id':str(uuid.uuid5(namespace=constants.NAMESPACE, name=str(pick.playerId))),
                    'team_id':str(uuid.uuid5(namespace=constants.NAMESPACE, name=team_name)),
                    'year':year,
                    'round':pick.round_num,
                    'pick':pick.round_pick
                }
            )

        for team in league.teams:
            team_name = team.owners[0]['firstName']
            if team_name == 'The':
                team_name = 'Klapp'
            elif team_name == 'Noah ':
                team_name = 'Noah'

            data_teams.append(
                    {
                        'team_id':str(uuid.uuid5(namespace=constants.NAMESPACE, name=team_name)),
                        'team_name':team_name
                    }
                )
            
        for player in players:
            if player['defaultPositionId'] not in [1, 2, 3, 4, 5, 16]:
                continue

            data_players.append(
                {
                    'player_id':str(uuid.uuid5(namespace=constants.NAMESPACE, name=str(player['id']))),
                    'player_name':player['fullName'],
                    'position':constants.DEFAULT_POSITION_MAP[player['defaultPositionId']]
                }
            )

        for week in box_scores.keys():
            matchups = box_scores[week]

            for matchup in matchups:
                matchup_id = str(uuid.uuid4())

                home_team = matchup.home_team.owners[0]['firstName']
                if home_team == 'The':
                    home_team = 'Klapp'
                elif home_team == 'Noah ':
                    home_team = 'Noah'

                if matchup.away_team == 0:
                    away_team = 'Bye'
                else:
                    away_team = matchup.away_team.owners[0]['firstName']
                if away_team == 'The':
                    away_team = 'Klapp'
                elif away_team == 'Noah ':
                    away_team = 'Noah'

                data_matchups.append(
                    {
                        'matchup_id':matchup_id,
                        'year':year,
                        'week':week,
                        'matchup_type':matchup.matchup_type,
                        'playoff_flag':matchup.is_playoff,
                        'home_team_id':str(uuid.uuid5(constants.NAMESPACE, name=home_team)),
                        'home_score':matchup.home_score,
                        'away_team_id':str(uuid.uuid5(constants.NAMESPACE, name=away_team)),
                        'away_score':matchup.away_score
                    }
                )

                home_game_id = str(uuid.uuid4())
                data_games.append(
                    {
                        'game_id':home_game_id,
                        'matchup_id':matchup_id,
                        'team_id':str(uuid.uuid5(constants.NAMESPACE, name=home_team)),
                        'score':matchup.home_score,
                        'opp_score':matchup.away_score,
                        'win_flag':int(matchup.home_score > matchup.away_score),
                        'margin':round(matchup.home_score - matchup.away_score, 2)
                    }
                )

                away_game_id = str(uuid.uuid4())
                data_games.append(
                    {
                        'game_id':away_game_id,
                        'matchup_id':matchup_id,
                        'team_id':str(uuid.uuid5(constants.NAMESPACE, name=away_team)),
                        'score':matchup.away_score,
                        'opp_score':matchup.home_score,
                        'win_flag':int(matchup.away_score > matchup.home_score),
                        'margin':round(matchup.away_score - matchup.home_score, 2)
                    }
                )

                if year > 2018:
                    for player in matchup.home_lineup:
                        data_player_games.append(
                            {
                                'player_game_id':str(uuid.uuid4()),
                                'matchup_id':matchup_id,
                                'game_id':home_game_id,
                                'team_id':str(uuid.uuid5(constants.NAMESPACE, name=home_team)),
                                'player_id':str(uuid.uuid5(constants.NAMESPACE, name=str(player.playerId))),
                                'points':player.points,
                                'projected_points':player.projected_points,
                                'slot_position':player.slot_position,
                                'active_status':player.active_status,
                                'bye_week_flag':player.on_bye_week
                            }
                        )

                    for player in matchup.away_lineup:
                        data_player_games.append(
                            {
                                'player_game_id':str(uuid.uuid4()),
                                'matchup_id':matchup_id,
                                'game_id':away_game_id,
                                'team_id':str(uuid.uuid5(constants.NAMESPACE, name=away_team)),
                                'player_id':str(uuid.uuid5(constants.NAMESPACE, name=str(player.playerId))),
                                'points':player.points,
                                'projected_points':player.projected_points,
                                'slot_position':player.slot_position,
                                'active_status':player.active_status,
                                'bye_week_flag':player.on_bye_week
                            }
                        )
    df_teams = pd.DataFrame(data_teams).drop_duplicates().reset_index(drop=True)
    df_drafts = pd.DataFrame(data_drafts)
    df_players = pd.DataFrame(data_players).drop_duplicates('player_id', keep='last').reset_index(drop=True)
    df_matchups = pd.DataFrame(data_matchups)
    df_games = pd.DataFrame(data_games)
    df_player_games = pd.DataFrame(data_player_games)


    return {'teams':df_teams, 'drafts':df_drafts, 'matchups':df_matchups, 'games':df_games, 'player_games':df_player_games, 'players':df_players}

In [33]:
data_dict = construct_dataframes(version=4)

In [22]:
def fetch_new_data(
        year: int,
        version: int,
        league_id=constants.LEAGUE_ID,
        espn_s2=constants.ESPN_S2,
        swid=constants.SWID
):
    league = League(league_id=league_id, year=year, espn_s2=espn_s2, swid=swid)
    players = league.espn_request.get_pro_players()

    matchups = {}
    for week in range(1, league.current_week + 1):
        box_scores = league.box_scores(week)
        matchups[week] = box_scores

        time.sleep(3)

    new_data = {
        'League':league,
        'Players':players,
        'Box Scores':matchups
    }

    data = espn_data.read_pickle_file(version=version)
    data[year] = new_data
    return data

data = fetch_new_data(year=2025, version=4)